In [ ]:
# Imports ----
import re
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
import pickle

In [ ]:
from g2fd.internal import *

In [ ]:
#| default_exp internal

In [ ]:
# 2020
year_string = '2020'

meta_path = './data/raw/GenomesToFields_G2F_data_2020/z._2020_supplemental_info/g2f_2020_field_metadata.csv'
phno_path = './data/raw/GenomesToFields_G2F_data_2020/a._2020_phenotypic_data/g2f_2020_phenotypic_clean_data.csv' 
wthr_path = './data/raw/GenomesToFields_G2F_data_2020/b._2020_weather_data/2020_weather_cleaned.csv'
soil_path = './data/raw/GenomesToFields_G2F_data_2020/c._2020_soil_data/g2f_2020_soil_data.csv'
mgmt_path = './data/raw/GenomesToFields_G2F_data_2020/z._2020_supplemental_info/g2f_2020_agronomic_information.csv'

meta = pd.read_csv(meta_path, encoding = "ISO-8859-1", low_memory=False)
phno = pd.read_csv(phno_path, encoding = "ISO-8859-1", low_memory=False)
wthr = pd.read_csv(wthr_path, encoding = "ISO-8859-1", low_memory=False)
soil = pd.read_csv(soil_path, encoding = "ISO-8859-1", low_memory=False)
mgmt = pd.read_csv(mgmt_path, encoding = "ISO-8859-1", low_memory=False)

In [ ]:
# load dicts for column renaming
meta_name_dict = mk_name_dict(name = 'meta')
phno_name_dict = mk_name_dict(name = 'phno')
soil_name_dict = mk_name_dict(name = 'soil')
wthr_name_dict = mk_name_dict(name = 'wthr')
mgmt_name_dict = mk_name_dict(name = 'mgmt')

# Rename
**Naming rules:**
- One dict for each input df
- Comment out anything that shouldn't be changed
- Upper_Upper_Unit_\$unit
- Upper_$number
- No special characters


In [ ]:
(find_unrecognized_columns(df = meta, dct = meta_name_dict),
find_unrecognized_columns(df = phno, dct = phno_name_dict),
find_unrecognized_columns(df = soil, dct = soil_name_dict),
find_unrecognized_columns(df = wthr, dct = wthr_name_dict),
find_unrecognized_columns(df = mgmt, dct = mgmt_name_dict))

([], [], [], [], [])

In [ ]:
meta = meta.rename(columns=meta_name_dict)
phno = phno.rename(columns=phno_name_dict)
soil = soil.rename(columns=soil_name_dict)
wthr = wthr.rename(columns=wthr_name_dict)
mgmt = mgmt.rename(columns=mgmt_name_dict)

# add indicator columns to help with debugging merge
meta['meta'] = True
phno['phno'] = True
soil['soil'] = True
wthr['wthr'] = True
mgmt['mgmt'] = True

In [ ]:
[e.shape for e in [meta, phno, soil, wthr, mgmt]]

[(26, 55), (18754, 41), (23, 30), (289867, 24), (142, 7)]

# Sanatize ID columns as needed


In [ ]:
wthr = sanitize_Experiment_Codes(
    df = wthr, 
    simple_renames = {
    }, 
    split_renames = {
        'NYH3_NYS1': ['NYS1', 'NYH3'],
        'TXH1_TXH3': ['TXH1', 'TXH3']
    })

# for some reason there are 2227 empty rows included...
wthr = wthr.loc[wthr.Experiment_Code.notna(), ]

In [ ]:
# confirm everything's okay
print(
  'meta', find_unrecognized_experiments(meta.Experiment_Code, return_all_exps=False), 
'\nphno', find_unrecognized_experiments(phno.Experiment_Code, return_all_exps=False),
'\nsoil', find_unrecognized_experiments(soil.Experiment_Code, return_all_exps=False),
'\nwthr', find_unrecognized_experiments(wthr.Experiment_Code, return_all_exps=False),
'\nmgmt', find_unrecognized_experiments(mgmt.Experiment_Code, return_all_exps=False),
'\nall ', find_unrecognized_experiments([], return_all_exps=True)
)  

meta [] 
phno [] 
soil [] 
wthr [] 
mgmt [] 
all  ['COH1', 'DEH1', 'GAH1', 'GAH2', 'GEH1', 'IAH1', 'IAH2', 'IAH3', 'IAH4', 'ILH1', 'INH1', 'MIH1', 'MNH1', 'MOH1', 'NCH1', 'NEH1', 'NEH2', 'NEH3', 'NYH2', 'NYH3', 'NYS1', 'OHH1', 'SCH1', 'TXH1', 'TXH2', 'TXH3', 'WIH1', 'WIH2', 'WIH3']


# Rearrange columns

In [ ]:
# separate static and dynamic values
sval = phno.merge(soil, how = 'outer')
sval = sval.merge(meta, how = 'outer') # This introduces 3 sites that have no data
# sval.shape # used to confirm nrow = #20574 + 3

# these tables are different enought we'll keep them separate
# mgmt
# unfortunately we need multiples because at least one field treats different passes differently
mgmt = phno.loc[:, ['Year', 'Experiment_Code', 'Range', 'Pass', 'Plot', 'phno']
               ].drop_duplicates().merge(mgmt, how = 'outer')
# confirm there are no rows in mgmt that are not in phno
temp = mgmt.loc[(~mgmt.phno & mgmt.mgmt), :]
if 0 != temp.shape[0]:
    print(temp)
else:
    mgmt = mgmt.loc[mgmt.mgmt.notna(), :].drop(columns = 'phno')


# wthr
# There's only ever one weather station so we have to worry about imputation but not duplicates

In [ ]:
# Set each id col to a string
for i in ['Year', 'Experiment_Code', 'Range', 'Pass', 'Plot']:
    sval[i] = sval[i].astype('string')
    mgmt[i]  =  mgmt[i].astype('string')
    
    if i not in ['Range', 'Pass', 'Plot']:
        wthr[i]  =  wthr[i].astype('string')

# Sanitize Non-ID columns



## Sanitization functions

The pattern to use is:
 1. Alter the dataframe
 1. Test the dataframe against expectations
 
The main tasks that need to be completed are:
 1. Identify values that can't be converted to the expected data type. The "find_unconvertable_" family of functions should be used. 
     1. `find_unconvertable_datetimes`
     
 1. For simple renaming (e.g. misspellings) or splitting non-tidy data into two rows ("entry1-entry2" -> "entry1", "entry2") use `sanitize_col` 
 1. Move values that are ambigous but pertain to data imputation to "Imputation_Notes" using `relocate_to_Imputation_Notes`
 1. If new columns need to be added (e.g. mgmt.Ingredient for parsed components of Product (e.g. elements) ) this should be accomplished with `safe_create_col`.
 1. Any one off changes should be accomplised manually. 
 1. Confirm columns match the expected types with `check_df_dtype_expectations`, and report mismatches. 


These steps should be completed for each dataframe in turn to minimize the cognitive load of the reader. 

## Sanitization: Column data type expectations
Note: to handle missing values some columns that would otherwise be ints are floats

In [ ]:
sval_col_dtypes = mk_dtype_dict(name = 'sval')
wthr_col_dtypes = mk_dtype_dict(name = 'wthr')
mgmt_col_dtypes = mk_dtype_dict(name = 'mgmt')

# Sanitization: Alter entries

## Static values (within season)

### Datetime containing columns

In [ ]:
# convert the date cols into datetime. Lean on pd.to_datetime() to infer the format, assume that each site uses the same format.

for e in ['Planted_Unit_Datetime', 
    'Harvested_Unit_Datetime', 
    'Anthesis_Unit_Datetime', 
    'Silking_Unit_Datetime', 
    'Recieved_Date_Unit_Datetime', 
    'Processed_Date_Unit_Datetime', 
    'Weather_Station_Placed_Unit_Datetime', 
    'Weather_Station_Removed_Unit_Datetime'
    ]:

    sval['Datetime_Temp'] = pd.to_datetime(np.nan)

    for code in list(sval.Experiment_Code.drop_duplicates()):
    # code = list(sval.Experiment_Code.drop_duplicates())[0]
        sval.loc[sval.Experiment_Code == code, 'Datetime_Temp'
                 ] = pd.to_datetime(sval.loc[sval.Experiment_Code == code, e])

    sval.loc[:, e] = sval.loc[:, 'Datetime_Temp'] 

sval = sval.drop(columns = 'Datetime_Temp')

In [ ]:
# convert types
for e in ['Alley_Length_Unit_Inches', 'Row_Spacing_Unit_Inches', 'Pounds_Needed_Soil_Moisture',
         'Anthesis_Unit_Days', 'Silking_Unit_Days', 'Kernels_Per_Plot']:
    err_list = find_unconvertable_numerics(df_col = sval[e], index = False)
    if err_list != []:
        print(e)
        print(err_list)
    else:
        sval[e] = sval[e].astype('float')

In [ ]:
# to bool
sval = sanitize_col(
    df = sval, 
    col = 'Discarded', 
    simple_renames= {
        'Yes':'True',
        'yes':'True'}, 
    split_renames= {})

# set missing to false
sval.loc[sval.Discarded.isna(), 'Discarded'] = 'False'
sval.Discarded = sval.Discarded.map({'True': True, 'False': False})

### Simple Columns

In [ ]:
# to bool
sval['phno'] = sval['phno'].astype('bool')
sval['soil'] = sval['soil'].astype('bool')
sval['meta'] = sval['meta'].astype('bool')

# to string
sval = cols_astype_string(
    df = sval, 
    col_list = [key for key in sval_col_dtypes.keys() if sval_col_dtypes[key] == 'string'])

sval.Year = year_string
sval.Year = sval.Year.astype('string')

### Check Success

In [ ]:
checkpoint = check_df_dtype_expectations(df = sval, dtype_dct = sval_col_dtypes)

if sum(checkpoint.Pass)/checkpoint.shape[0] == 1:
    pass
else:
    print(checkpoint.loc[~checkpoint.Pass, ]) #TODO add to template
    print()

120/120 Columns pass.


## Weather

### Datetime

In [ ]:
# 183 records from MNH1 are missing a value for time. 
# These are not useful since we don't know if the values are daily averages or taken at a specific timepoint.
wthr = wthr.loc[wthr.Time.notna(), :]

# unfortunately these missing values also force ints (e.g. days) to be floats
wthr.Year = wthr.Year.astype(float).astype(int).astype('string')
wthr.Month = wthr.Month.astype(float).astype(int).astype('string')
wthr.Day = wthr.Day.astype(float).astype(int).astype('string')

In [ ]:
# we use the fields in the df to make a consistent format
wthr = cols_astype_string(
    df = wthr, 
    col_list = ['Year', 'Month', 'Day', 'Time'])

wthr = sanitize_col(
    df = wthr,
    col = 'Time', 
    simple_renames= {}, 
    split_renames= {})

wthr['Datetime_Temp'] = wthr['Year']+'-'+wthr['Month']+'-'+wthr['Day']+' '+wthr['Time']

# convert types
err_list = find_unconvertable_datetimes(df_col=wthr['Datetime_Temp'], pattern='%Y-%m-%d %H:%M', index=False)
if err_list != []:
    print(err_list)
else:
    wthr.Datetime_Temp = pd.to_datetime(pd.Series(wthr.Datetime_Temp), errors='coerce')
    wthr.Datetime = wthr.Datetime_Temp
    wthr = wthr.drop(columns= 'Datetime_Temp')

### Simple Columns 

In [ ]:
# to string
wthr = cols_astype_string(
    df = wthr, 
    col_list = [key for key in wthr_col_dtypes.keys() if wthr_col_dtypes[key] == 'string'])

wthr.Year = year_string
wthr.Year = wthr.Year.astype('string')

### Check Success

In [ ]:
checkpoint = check_df_dtype_expectations(df = wthr, dtype_dct = wthr_col_dtypes)

if sum(checkpoint.Pass)/checkpoint.shape[0] == 1:
    pass
else:
    print(checkpoint.loc[~checkpoint.Pass, ])

24/24 Columns pass.


## Management

### Date_Datetime

In [ ]:
mgmt = relocate_to_Imputation_Notes(df = mgmt, col = 'Date_Datetime', val_list= ['fall 2019'])

In [ ]:
# convert types
err_list = find_unconvertable_datetimes(df_col=mgmt.Date_Datetime, pattern='%m/%d/%y', index=False)
if err_list != []:
    print(err_list)
else:
    mgmt.Date_Datetime = pd.to_datetime(pd.Series(mgmt.Date_Datetime), format = '%m/%d/%y', errors='coerce')

### Amount_Per_Acre

In [ ]:
mgmt = relocate_to_Imputation_Notes(df = mgmt, col = 'Amount_Per_Acre', val_list= ['variable rate according to soil sample'])

In [ ]:
# mgmt.loc[find_unconvertable_numerics(df_col = mgmt['Amount_Per_Acre'], index = True), ]

mgmt.loc[mgmt.Amount_Per_Acre == '2 qts', :]

,Year,Experiment_Code,Range,Pass,Plot,Application,Product,Date_Datetime,Amount_Per_Acre,Unit,mgmt,Imputation_Notes
77802,2020,SCH1,1.0,1,1,Broadcast,BicepMagnum II,2020-05-12,2 qts,NaN,True,NaN
77803,2020,SCH1,1.0,1,1,Broadcast,Glyphosate,2020-05-12,2 qts,NaN,True,NaN
77804,2020,SCH1,1.0,1,1,Broadcast,Atrazine,2020-06-10,2 qts,NaN,True,NaN
77805,2020,SCH1,1.0,2,2,Broadcast,BicepMagnum II,2020-05-12,2 qts,NaN,True,NaN
77806,2020,SCH1,1.0,2,2,Broadcast,Glyphosate,2020-05-12,2 qts,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
79711,2020,SCH1,8.0,2,637,Broadcast,Glyphosate,2020-05-12,2 qts,NaN,True,NaN
79712,2020,SCH1,8.0,2,637,Broadcast,Atrazine,2020-06-10,2 qts,NaN,True,NaN
79713,2020,SCH1,8.0,1,638,Broadcast,BicepMagnum II,2020-05-12,2 qts,NaN,True,NaN
79714,2020,SCH1,8.0,1,638,Broadcast,Glyphosate,2020-05-12,2 qts,NaN,True,NaN


In [ ]:
mask = (mgmt.Amount_Per_Acre == '2 qts')
mgmt.loc[mask, 'Unit'] = 'qts'
mgmt.loc[mask, 'Amount_Per_Acre'] = '2'

mask = (mgmt.Amount_Per_Acre == '1.5 qts')
mgmt.loc[mask, 'Unit'] = 'qts'
mgmt.loc[mask, 'Amount_Per_Acre'] = '1.5'

In [ ]:
# convert types
err_list = find_unconvertable_numerics(df_col = mgmt['Amount_Per_Acre'], index = False)
if err_list != []:
    print(err_list)
else:
    mgmt.Amount_Per_Acre = pd.to_numeric(mgmt.Amount_Per_Acre, errors='coerce')

### Simple Columns

In [ ]:
# to bool
mgmt['mgmt'] = mgmt['mgmt'].astype('bool')

# to string
for e in [ee for ee in ['Application', 'Product', 'Ingredient', 'Unit', 'Imputation_Notes'] if ee in mgmt.columns]:
    mgmt[e] = mgmt[e].astype('string')
    

mgmt.Year = year_string
mgmt.Year = mgmt.Year.astype('string')

### Check Success

In [ ]:
check_df_dtype_expectations(df = mgmt, dtype_dct = mgmt_col_dtypes)

12/12 Columns pass.


,Column,dtype,Expected_dtype,Pass
0,Year,string,string,True
1,Experiment_Code,string,string,True
2,Range,string,string,True
3,Pass,string,string,True
4,Plot,string,string,True
5,Application,string,string,True
6,Product,string,string,True
7,Date_Datetime,datetime64[ns],datetime64[ns],True
8,Amount_Per_Acre,float64,float64,True
9,Unit,string,string,True


# Publish



In [ ]:
write_out_pkl(obj = sval, path = './data/interim/'+year_string+'sval.pickle')
write_out_pkl(obj = wthr, path = './data/interim/'+year_string+'wthr.pickle')
write_out_pkl(obj = mgmt, path = './data/interim/'+year_string+'mgmt.pickle')